### Extraction des données brutes

In [30]:
import pandas as pd
from langdetect import detect

def charger_donnees(chemin):
    return pd.read_csv(chemin, sep=None, engine='python', encoding='utf-8-sig')

df = charger_donnees('export_articles_EGC_2004_2018.csv')
dfEditions = charger_donnees('edition_EGC.csv')

### Table Articles

In [2]:
def identifier_langue(texte):
    try:
        if pd.isna(texte) or str(texte).strip() == "":
            return "vide"
        return detect(str(texte))
    except:
        return "erreur"

### Table lien entre Articles et Authors

### Table Authors

In [25]:
def count_authors(auth_line):
    try:
        tab = auth_line.split(', ')
        return len(tab)
    except:
        return "erreur"

### Table Editions

In [31]:
def nb_articles_per_edition(df_art, df_edit): 
    counts = df_art['year'].value_counts().reset_index()
    counts.columns = ['year', 'nb_articles']

    df_result = df_edit.merge(counts, on='year', how='left')
    df_result['nb_articles'] = df_result['nb_articles'].fillna(0).astype(int)
    
    return df_result

### Ingestion des données dans des csv

In [32]:
# Articles DataFrame
# Traitement langue
dfArticles = df[['title', 'year']].copy()
dfArticles.insert(0, 'id_article', range(1, len(dfArticles) + 1))
dfArticles['language'] = df['title'].apply(identifier_langue)
# Nombre d'auteurs par article
dfArticles['Nb_authors'] = df['authors'].apply(count_authors)
dfArticles['Nb_male'] = dfArticles['Nb_authors']
dfArticles['Nb_female'] = dfArticles['Nb_authors']
dfArticles['theme'] = "pas encore"
dfArticles['keywords'] = "pas encore"

# Authors DataFrame
dfAuthors = pd.DataFrame()
dfAuthors['name'] = "pas encore"
dfAuthors['gender'] = "pas encore"
dfAuthors['id_article'] = "pas encore"

# Editions DataFrame
dfEditions = nb_articles_per_edition(df, dfEditions)

# Export
dfArticles.to_csv('articles.csv', index=False)
dfAuthors.to_csv('authors.csv', index=False)
dfEditions.to_csv('editions.csv', index=False)